In [ ]:
# fifth order correct WENO-Z reconstruction (from https://doi.org/10.1016/j.jcp.2007.11.038)
# given five cell averages v0 .. v4 (left to right),
# returns estimate from the left at interface b/w v2 and v3
# (one version enough - reverse order gives estimates from the right)
def weno5(v0, v1, v2, v3, v4):
    #3rd order estimates from smaller stencils (of size 3)
    p0 = v2/3 + 5*v3/6 - v4/6
    p1 = -v1/6 + 5*v2/6 + v3/3
    p2 = v0/3 - 7*v1/6 + 11*v2/6
    #smoothness indicators (to handle shocks):
    b0 = (13/12)*(v2 - 2*v3 + v4)**2 + (1/4)*(3*v2 - 4*v3 + v4)**2
    b1 = (13/12)*(v1 - 2*v2 + v3)**2 + (1/4)*(v1 - v3)**2
    b2 = (13/12)*(v0 - 2*v1 + v2)**2 + (1/4)*(v0 - 4*v1 + 3*v2)**2
    eps = 1e-7
    #weno-z improvements for extrema:
    tau = np.abs(b0 - b2)
    b0_z = (b0 + eps)/(b0 + tau + eps)
    b1_z = (b1 + eps)/(b1 + tau + eps)
    b2_z = (b2 + eps)/(b2 + tau + eps)
    #weights to combine smaller stencil ests:
    a0 = (3/10)*(b0_z)**(-2)
    a1 = (6/10)*(b1_z)**(-2)
    a2 = (1/10)*(b2_z)**(-2)
    return (a0*p0 + a1*p1 + a2*p2)/(a0 + a1 + a2)

# third order correct WENO
# takes three cell avgs v0 .. v2
# returns est from the left at interface b/w v1 and v2
def weno3(v0,v1,v2):
    #2nd order estimates (from size 2 stencils)
    p0 = 0.5*(v1 + v2)
    p1 = 1.5*v1 - 0.5*v0
    #smoothness indicators:
    b0 = (v2 - v1)**2
    b1 = (v1 - v0)**2
    eps = 1e-7
    #weights to combine smaller stencil ests:
    a0 = 1/((eps + b0[i])**2)
    a1 = 0.5/((eps + b1[i])**2)
    return (a0*p0 + a1*p1)/(a0 + a1)